In [1]:
from src.data.dataset_functions import get_dataframei_onlymods

modframe = get_dataframei_onlymods(3)

modframe.show(5)

providing ../data/raw/2022_place_canvas_history-000000000003.csv ...
not found. need to download ../data/raw/2022_place_canvas_history-000000000003.csv.gzip ...
downloading from https://placedata.reddit.com/data/canvas-history/2022_place_canvas_history-000000000003.csv.gzip to ../data/raw/2022_place_canvas_history-000000000003.csv.gzip
unpacking ../data/raw/2022_place_canvas_history-000000000003.csv.gzip into ../data/raw/2022_place_canvas_history-000000000003.csv
deleting ../data/raw/2022_place_canvas_history-000000000003.csv.gzip
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-86a2e06f-c8e4-46a6-8461-9e0a363ca9c9;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 71ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

+--------------------+---+----+---+----+---+-----------+
|             user_id| x1|  y1| x2|  y2|  t|pixel_color|
+--------------------+---+----+---+----+---+-----------+
|m8NEcPbf5XRV5ppeu...|298|1805|329|1839| 95|    #FFB470|
|LKS2u3QL2N3Olv7rn...|257|1736|296|1780|321|    #FFB470|
|q/Dk6lmcXm8bcDbNI...|298|1770|334|1803|  0|    #FFB470|
|HkR0yRQUJ1wsjh4Zo...|251|1805|296|1812|427|    #FFF8B8|
|7JiQyrONpFJphvBEP...|271|1835|296|1859|562|    #FFF8B8|
+--------------------+---+----+---+----+---+-----------+



In [2]:
from src.data.dataset_functions import get_dataframei

dataFrame = get_dataframei(3)

providing ../data/raw/2022_place_canvas_history-000000000003.csv ...
../data/raw/2022_place_canvas_history-000000000003.csv is already in data/raw


In [5]:
from src.features.feature_functions import get_latestpixels_from_box
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import functools
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Empty_Dataframe').getOrCreate()

#Mergen zweier DataFrames
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

#Ja, datacollect ist ineffizient ABER es gibt nicht so viele Zensuren, daher sollte das halb so wild sein!
mods = modframe.drop('pixel_color').drop('user_id').collect()

emp_RDD = spark.sparkContext.emptyRDD()

# Create an expected schema
columns = StructType([StructField('x',
                                  IntegerType(), True),
                    StructField('y',
                                IntegerType(), True),
                    StructField('user_id',
                                StringType(), True),
                    StructField('t',
                                LongType(), True),
                    StructField('pixel_color',
                                StringType(), True)])


censoredData = spark.createDataFrame(data = emp_RDD,
                                    schema = columns)

#DataFrame für alle gemachten Zensuren mit den betroffenen Pixel generieren
for row in mods:
    min_x,min_y,max_x,max_y,tz = list(row)
    censoredData = unionAll([censoredData,get_latestpixels_from_box(dataFrame,min_x,min_y,max_x,max_y,tz)])

In [6]:
censoredData.show(5)
censoredData.count()

+---+----+--------------------+---+-----------+
|  x|   y|             user_id|  t|pixel_color|
+---+----+--------------------+---+-----------+
|299|1812|QH6feXGSBIZObcDe/...| 73|    #FFB470|
|300|1805|RFAixfEVBuu5IBfU2...|  6|    #FFFFFF|
|300|1806|q5z+vy8G2Xu6bvO4P...|  7|    #00A368|
|300|1807|d4rv9KUGfHHVSo+Kp...| 36|    #FF99AA|
|301|1807|+3U9zAP3LWm2kMnJP...| 50|    #FF99AA|
+---+----+--------------------+---+-----------+
only showing top 5 rows



372

<div style="color:#4e31ce">

So bis hier wurde die Funktion nun für den GESAMTEN Datensatz verallgemeinert....

Jetzt muss DataFrame mit user_ids für Bots generiert werden und dann anhand eines Joins geschaut werden, welche Pixel zu Bots gehören.

Dann kann count() für das DataFrame mit den Bots angewendet werden und count() für die Ausgangstabelle mit den von der Zensur betroffenen
Pixeln, sodass das eine durch das andere geteilt werden kann um entsprechend das Verhältnis rauszubekommen!
</div>